<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/CNN_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra # เป็นการ import numpy เพื่อเปิดใช้ฟังก์ชันในการทำงาน
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) # เป็น library ในการจัดการ dataframe
import tensorflow as tf #library สำหรับสร้าง machine learning models แบบ open source จาก Google
import datetime,os # datetime ใช้ในการจัด format ของวัน-เวลาให้อยู่ในรูปแบบที่ต้องการ 
import math # เรียกใช้ module เพื่อคำนวณค่าต่างๆ 
import matplotlib.pyplot as plt # เป็น module พื้นฐานที่ใช้ในการสร้างกราฟของ python
import seaborn as sns # เป็นฟังก์ชันเสริมที่ใช้สร้างกราฟให้มีความสวยงามมากขึ้น
from sklearn.model_selection import train_test_split # เป็น library ในการพัฒนาโปรแกรมโดยใช้การเรียนรู้ของเครื่อง

In [3]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [5]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [6]:
#loading dataframes # เอาที่อยู่ของไฟล์เชื่อมกับที่อยู่ของไฟล์เรา โดยกำหนดชื่่อไฟล์ที่เราต้องการ
train = pd.read_csv(path+"/train.csv")
validation = pd.read_csv(path+"/validation.csv")
test = pd.read_csv(path+"/boneage-test-dataset.csv")

In [7]:
#appending file extension to id column for both training and testing dataframes
# ต่อท้าย data ใน column 'id' ด้วย .png
train['id'] = train['id'].apply(lambda x: str(x)+'.png') # ทำการเปลี่ยน type ใน column 'id' จาก int  เป็น string เพื่อให้สามารถต่อท้ายด้วย .png ได้ # ฟังก์ชันแลมบ์ดาเป็นฟังก์ชันที่ไม่ระบุตัวตนขนาดเล็ก
validation['id'] = validation['id'].apply(lambda x: str(x)+'.png') 
test['id'] = test['id'].apply(lambda x: str(x)+'.png') 

train.head() # แสดงค่าของ train_df บางส่วน

,id,boneage,male
0,1377.png,180,False
1,1378.png,12,False
2,1379.png,94,False
3,1380.png,120,True
4,1381.png,82,False


In [8]:
#finding out the number of male and female children in the dataset
#creating a new column called gender to keep the gender of the child as a string
train['male'] = train['male'].apply(lambda x: 'male' if x else 'female') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
validation['male'] = validation['male'].apply(lambda x: 'male' if x else 'female') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
test['male'] = test['male'].apply(lambda x: 'male' if x else 'female')

In [9]:
train = pd.DataFrame(train)
val = pd.DataFrame(validation)
test = pd.DataFrame(test)

In [10]:
train.rename(columns={'male':'gender'},inplace=True)
val.rename(columns={'male':'gender'},inplace=True)
test.rename(columns={'male':'gender'},inplace=True)

In [11]:
mean_bone_age_train = train['boneage'].mean() 
std_bone_age_train = train['boneage'].std()
train['bone_age_z'] = (train['boneage'] - mean_bone_age_train)/(std_bone_age_train)
print(train.head())

         id  boneage  gender  bone_age_z
0  1377.png      180  female    1.278306
1  1378.png       12  female   -2.796338
2  1379.png       94  female   -0.807524
3  1380.png      120    male   -0.176924
4  1381.png       82  female   -1.098570


In [12]:
mean_bone_age_val = val['boneage'].mean() 
std_bone_age_val = val['boneage'].std()
val['bone_age_z'] = (val['boneage'] - mean_bone_age_val)/(std_bone_age_val)
print(val.head())

          id  boneage  gender  bone_age_z
0  10186.png       10    male   -2.864366
1  10758.png       15  female   -2.742400
2  10860.png      180  female    1.282468
3  10877.png      180  female    1.282468
4  11008.png      180  female    1.282468


In [13]:
#import matplotlib.image as mpimg #นำเข้า library ที่ใช้อ่านรูป
#for filename, boneage, gender in train_df[['id','boneage','gender']].sample(10).values: #เป็นคำสั่งวนลูปเพื่อแสดงภาพ และข้อมูลของภาพที่เก็บไว้ใน Train_df (ชื่อ อายุ เพศ)
#    img = mpimg.imread(os.path.join(path,'boneage-training-dataset-5261/')+ filename) # เป็นคำสั่งที่ใช้ในการนำรูปเข้ามา
#    plt.imshow(img) # แสดงรูปภาพ
#    plt.title('Image name:{}  Bone age: {} years  Gender: {}'.format(filename, boneage/12, gender)) # แสดงหัวเรื่องโดยบอกถึง ชื่อ อายุ(ปี) และเพศ ของแต่ละภาพ
#    plt.axis('off')
#    plt.show()

In [14]:
#library required for image preprocessing 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from  keras.applications.xception import preprocess_input

In [15]:
#reducing down the size of the image 
img_size = 256 # ลดขนาดรูปภาพ

In [16]:
train_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
val_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

In [17]:
#train data generator
train_generator = train_data_generator.flow_from_dataframe(
    dataframe = train,
    directory = (os.path.join(path,'boneage_training_dataset/')),
    x_col= 'id',
    y_col= 'bone_age_z',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode= 'raw',
    #flip_vertical = True,
    color_mode = 'rgb',
    target_size = (img_size, img_size))

Found 10089 validated image filenames.


In [18]:
train_generator

In [19]:
#validation data generator
val_generator = val_data_generator.flow_from_dataframe(
    dataframe = val,
    directory = (os.path.join(path,'boneage_validation_dataset/')),
    x_col = 'id',
    y_col = 'bone_age_z',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'raw',
    #flip_vertical = True,
    color_mode = 'rgb',
    target_size = (img_size, img_size))

Found 2522 validated image filenames.


In [20]:
#test data generator
test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_data_generator.flow_from_directory(
    directory = (os.path.join(path,'boneage-test-dataset/')),
    target_size = (img_size,img_size),
    shuffle = True,
    class_mode = 'binary',classes=['.'],
    color_mode = 'rgb')

Found 200 images belonging to 1 classes.


In [21]:
train_X,train_Y = next(train_data_generator.flow_from_dataframe( 
                            train_df, 
                            directory = (os.path.join(path,'boneage_training_dataset/')),
                            x_col = 'id',
                            y_col = 'bone_age_z', 
                            target_size = (img_size, img_size),
                            batch_size = 10090,
                            class_mode = 'raw'
                            )) 

NameError: ignored

In [ ]:
test_X, test_Y = next(val_data_generator.flow_from_dataframe( 
                            val_df, 
                            directory = (os.path.join(path,'boneage_validation_dataset/')),
                            x_col = 'id',
                            y_col = 'bone_age_z', 
                            target_size = (img_size, img_size),
                            batch_size = 2523,
                            class_mode = 'raw'
                            )) 

#Model

In [22]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [23]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [24]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [25]:
# Define relevant variables for the ML task
START_EPOCH=0
NUM_EPOCHS=60
LR=0.0001
PATIENCE=2
BATCH_SIZE=20
NUM_WORKERS=0
NUM_GPUS=1
GPUS=0

In [26]:
# Device will determine whether to run the training on GPU or CPU.
#device = torch.device('cuda' if torch.cuda.is_available() else 'GPU')


In [27]:
if torch.cuda.is_available():
    device = torch.device('cpu')
else:
    device = torch.device('gpu')

In [28]:
import torchvision.datasets as datasets
from torch.autograd import Variable

In [ ]:
# Dataloaders
train_transform = transforms.Compose([transforms.Resize((256, 256)),
                               transforms.RandomAffine(
                                   20, translate=(0.2, 0.2),
                                   scale=(1, 1.2)),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor()])

val_transform = transforms.Compose([transforms.Resize((256, 256)),
                               transforms.ToTensor()])

In [29]:
# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_generator,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)

train_loader

In [30]:
val_loader = torch.utils.data.DataLoader(dataset = val_generator,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)
val_loader

In [31]:
test_loader = torch.utils.data.DataLoader(dataset = test_generator,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)
test_loader

In [ ]:
Sample_X_data = Variable(torch.Tensor(train_X))

In [ ]:
Sample_X_data.info()

In [ ]:
Sample_Y_data = Variable(torch.Tensor(test_Y))

In [32]:
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.layer = torch.nn.Linear(1, 1)

   def forward(self, x):
       x = self.layer(x)      
       return x

net = Net()
print(net)

Net(
  (layer): Linear(in_features=1, out_features=1, bias=True)
)


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.image_features_ = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=2, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Conv2d(16, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Conv2d(128, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Conv2d(256, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(128, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.numeric_features_ = nn.Sequential(
             nn.Linear(9, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64, 64*3),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64*3, 64*3*3),
            nn.ReLU(inplace=True),
        )


    def forward(self, x,y):
        x = self.image_features_(x)
        x=x.view(-1, 64*3*3)
        y=self.numeric_features_(y)

        return x,y

In [ ]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
model = NeuralNetwork().to(device)
#optimizer=optim.Adam(model.parameters(),1e-3)
#loss_fn=nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
loss_func = torch.nn.MSELoss()
criterion = nn.L1Loss()


In [ ]:
def test_loop(dataloader, model, loss_func):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for imgs,numeric_features, price in dataloader:
            imgs=imgs.to(device)
            numeric_features=numeric_features.to(device)
            price = price.to(device)
            pred = model(imgs,numeric_features)
            test_loss += loss_func(pred, price).item()
            correct += (pred.argmax(1) == price).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_func, train_loader, val_loader):
    for epoch in range(1,n_epochs+1):
        loss_train = 0.0
        for imgs,numeric_features in train_loader:
            imgs=imgs.to(device)
            numeric_features=numeric_features.to(device)
            #price = price.to(device)
            output=model(imgs, numeric_features)

            
            loss=loss_func(output)
            
            #L2 Regularization
            l2_lambda=0.001
            l2_norm=sum(p.pow(2).sum() for p in model.parameters())
            loss=loss+l2_lambda*l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(), epoch,
                                                         loss_train / len(train_loader)))
            test_loop(dataloader=val_loader, model=model, loss_func=loss_func)

In [ ]:
training_loop(
n_epochs = 100,
optimizer = optimizer,
model = model,
loss_func = loss_func,
train_loader = train_loader,
val_loader=val_loader)

In [ ]:
import torch
import torch.nn as nn

class NN(nn.Module):
  def __init__(self):
      super(NN, self).__init__()
      
      self.layer1 = nn.Sequential(
          torch.nn.Conv2d(2,32,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(2,2))

      self.layer2 = nn.Sequential(
          torch.nn.Conv2d(32,64,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(2,2))
      
      self.layer3 = nn.Sequential(
          torch.nn.Conv2d(64,128,kernel_size=3,padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(2,2,padding=1))
      
      self.flattened_tensor = nn.Flatten()

      self.linear_layer = nn.Sequential(
          torch.nn.Linear(128, 64),
          torch.nn.ReLU(),
          torch.nn.Linear(64,4)
      )
    
  def forward(self, inp):
    conv_output = self.layer3(self.layer2(self.layer1(inp)))
    flattened_output = self.flattened_tensor(conv_output)
    
    transposed_matrix = torch.transpose(flattened_output, 0, 1)
    
    linear_output = self.linear_layer(transposed_matrix)
    return linear_output

model = NN()
print(model)
#output = model()

In [ ]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()

        # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.max_pool1(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out
net = Net()
print(net)

In [35]:
# Define Optimizer and Loss Function
optimizer = torch.optim.SGD(net.parameters(), lr=LR)
loss_func = torch.nn.MSELoss()

In [36]:
#model = Net()
criterion = nn.L1Loss()

#Training

In [41]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
def training_loop(n_epochs, optimizer, model, loss_func, train_loader, val_loader):
    for epoch in range(60):
	#Load in the data in batches using the train_loader object
        for images, labels in enumerate(train_loader):  
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
        
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, NUM_EPOCHS, loss.item()))

            
            


In [43]:
training_loop(
n_epochs = 100,
optimizer = optimizer,
model = net,
loss_func = loss_func,
train_loader = train_loader,
val_loader=val_loader)

AttributeError: ignored

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
#def train_one_epoch(epoch_index, tb_writer):


    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        #running_loss += loss.item()
        #if i % 1000 == 999:
        #    last_loss = running_loss / 1000 # loss per batch
        #    print('  batch {} loss: {}'.format(i + 1, last_loss))
        #    tb_x = epoch_index * len(train_loader) + i + 1
        #    tb_writer.add_scalar('Loss/train', last_loss, tb_x)
        #    running_loss = 0.
print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, NUM_EPOCHS, loss.item()))
    #return last_loss

In [ ]:
images = images.to(device)
images

In [ ]:
inputs = Variable(Sample_X_data)
outputs = Variable(Sample_Y_data)
for i in range(60):
   prediction = Net(inputs)
   loss = loss_func(prediction, outputs) 
   optimizer.zero_grad()
   loss.backward()        
   optimizer.step()       

   if i % 10 == 0:
       # plot and show learning process
       plt.cla()
       plt.scatter(Sample_X_data.data.numpy(), Sample_Y_data.data.numpy())
       plt.plot(Sample_X_data.data.numpy(), prediction.data.numpy(), 'r-', lw=2)
       plt.text(0.5, 0, 'Loss=%.4f' % loss.data.numpy(), fontdict={'size': 10, 'color':  'red'})
       plt.pause(0.1)

plt.show()